In [12]:
from tkinter import *
import random
import ctypes

class setting:
    WIDTH = 1080
    HEIGHT = 720
    GRID_SIZE = 6
    CELL_COUNT = GRID_SIZE ** 2
    MINES_COUNT = (CELL_COUNT) // 4

    def height_prct(percentage):
        return (setting.HEIGHT / 100) * percentage

    def width_prct(percentage):
        return (setting.WIDTH / 100) * percentage
    
class Cell:
    all = []
    cell_count = setting.CELL_COUNT
    cell_count_label = None
    
    def __init__(self,x, y, mine=False):
        self.mine = mine
        self.is_opened = False
        self.mine_candidate = False
        self.cell_btn = None
        self.x = x
        self.y = y

        Cell.all.append(self)

    def create_btn(self, location):
        btn = Button(location, width=12, height=4,)
        btn.bind('<Button-1>', self.left_click_actions)
        btn.bind('<Button-3>', self.right_click_actions)
        self.cell_btn = btn

    @staticmethod
    def create_cell_count_label(location):
        lbl = Label(location, 
                    bg='black', 
                    fg='white', 
                    text=f"Cells Left:{Cell.cell_count}", 
                    font=("", 30))
        Cell.cell_count_label = lbl

    def left_click_actions(self, event):
        if self.mine:
            self.show_mine()
            
        else:
            if self.surrounded_cells_mines_length == 0:
                for cell_obj in self.surrounded_cells:
                    cell_obj.show_cell()
            self.show_cell()

            if Cell.cell_count == setting.MINES_COUNT:
                
                if ctypes.windll.user32.MessageBoxW(0, 'Congratulations! You won the game!', 'OK', 0):
                    root.destroy()
                
    def get_cell(self, x,y):
        for cell in Cell.all:
            if cell.x == x and cell.y == y:
                return cell

    @property
    def surrounded_cells(self):
        cells = [self.get_cell(self.x - 1, self.y -1),
                 self.get_cell(self.x - 1, self.y),
                 self.get_cell(self.x - 1, self.y + 1),
                 self.get_cell(self.x, self.y - 1),
                 self.get_cell(self.x + 1, self.y - 1),
                 self.get_cell(self.x + 1, self.y),
                 self.get_cell(self.x + 1, self.y + 1),
                 self.get_cell(self.x, self.y + 1)]

        cells = [cell for cell in cells 
                 if cell is not None]
        return cells

    @property
    def surrounded_cells_mines_length(self):
        counter = 0
        for cell in self.surrounded_cells:
            if cell.mine:
                counter += 1

        return counter

    def show_cell(self):
        if not self.is_opened:
            Cell.cell_count -= 1
            self.cell_btn.configure(text=self.surrounded_cells_mines_length)

            if Cell.cell_count_label:
                Cell.cell_count_label.configure(text=f"Cells Left:{Cell.cell_count}")

            self.cell_btn.configure(bg='SystemButtonFace')

        self.is_opened = True

    def show_mine(self):
        self.cell_btn.configure(bg='red')
        
        if ctypes.windll.user32.MessageBoxW(0, 'You clicked on a mine', 'Game Over', 0):
            root.destroy()        
        
    def right_click_actions(self, event):
        if not self.mine_candidate:
            self.cell_btn.configure(bg='orange')
            self.mine_candidate = True
            
        else:
            self.cell_btn.configure(bg='SystemButtonFace')
            self.mine_candidate = False

    @staticmethod
    def randomize_mines():
        picked_cells = random.sample(Cell.all, 
                                     setting.MINES_COUNT)
        
        for picked_cell in picked_cells:
            picked_cell.mine = True

    def __repr__(self):
        return f"Cell({self.x}, {self.y})"
    
root = Tk()
root.configure(bg="black")
root.geometry(f'{setting.WIDTH}x{setting.HEIGHT}')
root.title("Minesweeper Game")
root.resizable(False, False)

top_frame = Frame(root, 
                  bg='black', 
                  width=setting.WIDTH, 
                  height=setting.height_prct(25))
top_frame.place(x=0, y=0)

game_title = Label(top_frame, 
                   bg='black', 
                   fg='white', 
                   text='Minesweeper Game', 
                   font=('', 48))

game_title.place(x=setting.width_prct(25), 
                 y=0)

left_frame = Frame(root, 
                   bg='black', 
                   width=setting.width_prct(25), 
                   height=setting.height_prct(75))
left_frame.place(x=0, 
                 y=setting.height_prct(25))

center_frame = Frame(root, 
                     bg='black', 
                     width=setting.width_prct(75), 
                     height=setting.height_prct(75))
center_frame.place(x=setting.width_prct(25), 
                   y=setting.height_prct(25))

for x in range(setting.GRID_SIZE):
    for y in range(setting.GRID_SIZE):
        c = Cell(x, y)
        c.create_btn(center_frame)
        c.cell_btn.grid(column=x, row=y)
        
Cell.create_cell_count_label(left_frame)
Cell.cell_count_label.place(x=0, y=0)

Cell.randomize_mines()

root.mainloop()